<a href="https://colab.research.google.com/github/G-who0212/video_image_processing/blob/main/googlenet/googleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SIZE = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
        'drive/MyDrive/Cars Dataset/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        class_mode="sparse",
        batch_size=64
)

count=0
for image_batch, label_batch in train_generator:
#     print(label_batch)
    print(image_batch[0])
    break

Found 3355 images belonging to 7 classes.
[[[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.99896884 1.         0.9979377 ]
  [0.997147   1.         1.        ]
  [0.99889416 1.         1.        ]]

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.99950886 1.         0.99901766]
  [0.99660707 1.         1.        ]
  [0.9994342  1.         1.        ]]

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [0.9999644  0.99996054 1.        ]
  [0.9966059  0.9984523  1.        ]
  [0.99997413 1.         1.        ]]

 ...

 [[1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  ...
  [1.         1.         1.        ]
  [1.         1.         1.        ]
  [1.         1.         1.        ]]

 [[1.         1.         1.     

In [ ]:
class_names = list(train_generator.class_indices.keys())
class_names

['Audi',
 'Hyundai Creta',
 'Mahindra Scorpio',
 'Rolls Royce',
 'Swift',
 'Tata Safari',
 'Toyota Innova']

In [ ]:
test_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        'drive/MyDrive/Cars Dataset/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        class_mode="sparse",
        batch_size=64
)

Found 813 images belonging to 7 classes.


In [ ]:
for image_batch, label_batch in test_generator:
    print(image_batch[0])
    break

[[[0.9885715  0.9885715  0.9885715 ]
  [0.9892497  0.9892497  0.9892497 ]
  [0.989928   0.989928   0.989928  ]
  ...
  [0.9843138  0.9843138  0.9843138 ]
  [0.9847091  0.9847091  0.9847091 ]
  [0.98823535 0.98823535 0.98823535]]

 [[0.98823535 0.98823535 0.98823535]
  [0.98823535 0.98823535 0.98823535]
  [0.98823535 0.98823535 0.98823535]
  ...
  [0.9843138  0.9843138  0.9843138 ]
  [0.98538727 0.98538727 0.98538727]
  [0.98823535 0.98823535 0.98823535]]

 [[0.98823535 0.98823535 0.98823535]
  [0.98823535 0.98823535 0.98823535]
  [0.98823535 0.98823535 0.98823535]
  ...
  [0.9843138  0.9843138  0.9843138 ]
  [0.9860655  0.9860655  0.9860655 ]
  [0.98823535 0.98823535 0.98823535]]

 ...

 [[0.8096536  0.8096536  0.80626833]
  [0.8510597  0.8510597  0.8510597 ]
  [0.7656395  0.7698609  0.7677502 ]
  ...
  [0.7231133  0.7231133  0.73095644]
  [0.69462764 0.69462764 0.7024708 ]
  [0.666142   0.666142   0.6739851 ]]

 [[0.83406985 0.83406985 0.8320411 ]
  [0.8449556  0.8449556  0.8449556 ]


In [ ]:
def inception(x,
              filters_1x1,
              filters_3x3_reduce,
              filters_3x3,
              filters_5x5_reduce,
              filters_5x5,
              filters_pool):
  path1 = layers.Conv2D(filters_1x1, (1, 1), padding='same',    activation='relu')(x)
  path2 = layers.Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
  path2 = layers.Conv2D(filters_3x3, (1, 1), padding='same', activation='relu')(path2)
  path3 = layers.Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
  path3 = layers.Conv2D(filters_5x5, (1, 1), padding='same', activation='relu')(path3)
  path4 = layers.MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
  path4 = layers.Conv2D(filters_pool, (1, 1), padding='same', activation='relu')(path4)
  return tf.concat([path1, path2, path3, path4], axis=3)

In [ ]:
inp = layers.Input(shape=(32, 32, 3))
input_tensor = layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=(32, 32, 3))(inp)
x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(input_tensor)
x = layers.MaxPooling2D(3, strides=2)(x)
x = layers.Conv2D(64, 1, strides=1, padding='same', activation='relu')(x)
x = layers.Conv2D(192, 3, strides=1, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(3, strides=2)(x)
x = inception(x, filters_1x1=64, filters_3x3_reduce=96, filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool=32)
x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool=64)
x = layers.MaxPooling2D(3, strides=2)(x)
x = inception(x, filters_1x1=192, filters_3x3_reduce=96, filters_3x3=208, filters_5x5_reduce=16, filters_5x5=48, filters_pool=64)
aux1 = layers.AveragePooling2D((5, 5), strides=3)(x)
aux1 =layers.Conv2D(128, 1, padding='same', activation='relu')(aux1)
aux1 = layers.Flatten()(aux1)
aux1 = layers.Dense(1024, activation='relu')(aux1)
aux1 = layers.Dropout(0.7)(aux1)
aux1 = layers.Dense(10, activation='softmax')(aux1)
x = inception(x, filters_1x1=160, filters_3x3_reduce=112, filters_3x3=224, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=256, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
x = inception(x, filters_1x1=112, filters_3x3_reduce=144, filters_3x3=288, filters_5x5_reduce=32, filters_5x5=64, filters_pool=64)
aux2 = layers.AveragePooling2D((5, 5), strides=3)(x)
aux2 =layers.Conv2D(128, 1, padding='same', activation='relu')(aux2)
aux2 = layers.Flatten()(aux2)
aux2 = layers.Dense(1024, activation='relu')(aux2)
aux2 = layers.Dropout(0.7)(aux2) 
aux2 = layers.Dense(10, activation='softmax')(aux2)
x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
x = layers.MaxPooling2D(3, strides=2)(x)
x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
x = inception(x, filters_1x1=384, filters_3x3_reduce=192, filters_3x3=384, filters_5x5_reduce=48, filters_5x5=128, filters_pool=128)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)
out = layers.Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs = inp, outputs = [out, aux1, aux2])

In [ ]:
model.compile(optimizer='adam', 
              loss=[losses.sparse_categorical_crossentropy,
                    losses.sparse_categorical_crossentropy,
                    losses.sparse_categorical_crossentropy],
              loss_weights=[1, 0.3, 0.3],
              metrics=['accuracy'])
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=40)

Epoch 1/40
53/53 [==============================] - 933s 17s/step - loss: 3.1199 - dense_9_loss: 1.9452 - dense_6_loss: 1.9540 - dense_8_loss: 1.9616 - dense_9_accuracy: 0.2328 - dense_6_accuracy: 0.2259 - dense_8_accuracy: 0.2289 - val_loss: 2.9878 - val_dense_9_loss: 1.8675 - val_dense_6_loss: 1.8690 - val_dense_8_loss: 1.8652 - val_dense_9_accuracy: 0.2337 - val_dense_6_accuracy: 0.2337 - val_dense_8_accuracy: 0.2337
Epoch 2/40
53/53 [==============================] - 619s 12s/step - loss: 3.0088 - dense_9_loss: 1.8754 - dense_6_loss: 1.8852 - dense_8_loss: 1.8929 - dense_9_accuracy: 0.2343 - dense_6_accuracy: 0.2489 - dense_8_accuracy: 0.2396 - val_loss: 2.9655 - val_dense_9_loss: 1.8611 - val_dense_6_loss: 1.8201 - val_dense_8_loss: 1.8613 - val_dense_9_accuracy: 0.2448 - val_dense_6_accuracy: 0.2927 - val_dense_8_accuracy: 0.2448
Epoch 3/40
53/53 [==============================] - 617s 12s/step - loss: 2.9588 - dense_9_loss: 1.8528 - dense_6_loss: 1.8317 - dense_8_loss: 1.8553 - 

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train','Val'])
axs[1].plot(history.history['dense_4_accuracy'])
axs[1].plot(history.history['val_dense_4_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
scores = model.evaluate(test_generator)

In [ ]:
scores